In [1]:
# Third party packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import display

# Adding path
from os.path import abspath
import sys
sys.path.insert(0, abspath('..'))

# 
from query_request import *
from query_def import *
import credentials
# from sqlalchemy import create_engine

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth=None


In [2]:
# Demo for LDM

# Upload dataset
# downlaod dataset

In [3]:
# Demo for dragoman

# [Creating Endpoint](../configs/docker_command_icews.sh)

In [4]:
# create rdfmts
from DeTrusty import run_query
from DeTrusty.Molecule.MTCreation import create_rdfmts
from DeTrusty.Molecule.MTManager import ConfigFile

# Creating endpoints dictionary
endpoints = dictfrmjson('../configs/endpoints_sample.json')
print(endpoints)

# Creating RDF molecule template (rdfmts) from endpoints
# log_rdfmts = create_rdfmts(endpoints, '../configs/rdfmts.json')

# if already rdfmts has been created 
config_private = ConfigFile('../configs/rdfmts_private.json')
config_public = ConfigFile('../configs/rdfmts_public.json')

Total time taken by dictfrmjson function: 4.744529724121094e-05
{'https://labs.tib.eu/sdm/worldbank_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/icews_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/lei_endpoint/sparql': {}, 'https://skynet.coypu.org/coypu-internal': {'username': 'Your UserName', 'password': 'Your Password'}}


## Endpoints Status

In [5]:
# Demo query to all available enpoints

# demo query
query_test = """SELECT * WHERE{ ?Subject a ?Concept }LIMIT 1000"""


# creating request for cmemc
display("############### Cmemc Endpoint ###############")
cmemc_request = SPARQLRequest(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
# executing query and printing response
cmemc_request.execute(query_test)
buf = StringIO(cmemc_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for skynet
display("############### Skynet Endpoint ###############")
skynet_request = SPARQLRequest(skynet_endpoint, skynet_user, skynet_pass, 'basic')
# executing query and printing response
skynet_request.execute(query_test)
buf = StringIO(skynet_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for worldbank
display("############### Worldbank Endpoint ###############")
worldbank_request = SPARQLRequest(worldbank_endpoint)
# executing query and printing response
worldbank_request.execute(query_test)
display(worldbank_request.response.status_code)
buf = StringIO(worldbank_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for LEI
display("############### LEI Endpoint ###############")
lei_request = SPARQLRequest(lei_endpoint)
# executing query and printing response
lei_request.execute(query_test)
buf = StringIO(lei_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for ICEWS
display("############### ICEWS Endpoint ###############")
icews_request = SPARQLRequest(icews_endpoint)
# executing query and printing response
icews_request.execute(query_test)
buf = StringIO(icews_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))


'############### Cmemc Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.2840769290924072


,Subject,Concept
995,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/2000/01/rdf-schema#Resource
996,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/ns/shacl#PropertyShape
997,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
998,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2000/01/rdf-schema#Resource
999,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2002/07/owl#DatatypeProperty


'############### Skynet Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.05991649627685547


...

Passed Query Status: 200
Total time taken by execute function: 0.06162428855895996


,Subject,Concept
995,https://data.coypu.org/lei/2138006HHBEMF1TGWU29-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
996,https://data.coypu.org/lei/2138006HHUG8LC4UD387-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
997,https://data.coypu.org/lei/2138006HHXWLEK5WVK02-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
998,https://data.coypu.org/lei/2138006HI1P4JTM2WM22-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
999,https://data.coypu.org/lei/2138006HI5AF5E1P4T95-LA,https://www.gleif.org/ontology/Base/PhysicalAddress


'############### ICEWS Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.06310915946960449


,Subject,Concept
995,https://data.coypu.org/icews/36616020,https://schema.coypu.org/global#Conflict
996,https://data.coypu.org/icews/36616021,https://schema.coypu.org/global#Conflict
997,https://data.coypu.org/icews/36616024,https://schema.coypu.org/global#Conflict
998,https://data.coypu.org/icews/36616025,https://schema.coypu.org/global#Conflict
999,https://data.coypu.org/icews/36616026,https://schema.coypu.org/global#Conflict


In [ ]:
print(query_1_fdq_desc)
print(query_1_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_1_fdq_ex))
fdquery.post_query(query_1_fdq_ex)
fdquery.to_save('../../query_results/', filename='query_1_fdq_ex')

In [ ]:
query_1_fdq

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

df = pd.read_csv('../../query_results/query_1_fdq_ex.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
try:
    df.to_sql('query_1_fdq_ex', con=engine, if_exists='fail', index=False)
except Exception as e:
    print('Exeception: {}',e)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

query_1_fdq_ex_sql = """SELECT country_code, year, AVG(value) as carbon_emission, COUNT(disaster) AS no_of_disasters
FROM `query_1_fdq_ex`
WHERE year=year_dis
GROUP By country_code, year
Order By  year ASC
"""

df = pd.read_sql(query_1_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [ ]:
def carbon_emission_group(carbon_emission):
    if carbon_emission/1000000 < 0.25:
        return '0.00-0.25'
    elif carbon_emission/1000000 >= 0.25 and carbon_emission/1000000 < 0.5:
        return '0.25-0.5'
    elif carbon_emission/1000000 >= 0.5 and carbon_emission/1000000 < 0.75:
        return '0.50-0.75'
    else:
        return '> 0.75'
    

df = pd.read_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8')
df['carbon_emission(1e6)'] = df['carbon_emission'].apply(carbon_emission_group)

df.head(10)

In [ ]:

fig = px.scatter_geo(df, locations="country_code", color="carbon_emission(1e6)",
                     hover_data=['carbon_emission'], size="no_of_disasters",
                     animation_frame="year",
                     projection="equirectangular", scope='europe')
fig.update_layout(title_text='Carbon emission and the number of disaster events for various countries across years',\
                  title_font_color="red", title_font_family="Times New Roman", title_font_size=25)
fig.write_html('../../query_results/query_1_fdq_ex_sql_new.html')
fig.show()